In [4]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from collections import OrderedDict
import numpy as np



In [5]:
# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations including resizing to 299x299
transform = transforms.Compose([
    transforms.Resize((299, 299)),  # Adjusted input size
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load datasets with the above transformations
train_data = datasets.ImageFolder(root='/kaggle/input/recycle-with-split/recycle_split/train', transform=transform)
val_data = datasets.ImageFolder(root='/kaggle/input/recycle-with-split/recycle_split/test', transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

# Load InceptionV3 and modify for number of classes
model = models.inception_v3(pretrained=True)
model.aux_logits = True
num_features = model.fc.in_features
num_classes = 4
model.fc = nn.Linear(num_features, num_classes)
model.AuxLogits.fc = nn.Linear(model.AuxLogits.fc.in_features, num_classes)
model = model.to(device)

# Define label smoothing
class LabelSmoothingLoss(nn.Module):
    def __init__(self, smoothing=0.1):
        super(LabelSmoothingLoss, self).__init__()
        self.smoothing = smoothing

    def forward(self, pred, target):
        confidence = 1.0 - self.smoothing
        smooth_label = torch.full(size=(target.size(0), pred.size(1)), fill_value=self.smoothing / (pred.size(1) - 1)).to(device)
        smooth_label.scatter_(1, target.unsqueeze(1), confidence)
        return torch.mean(torch.sum(-smooth_label * torch.log_softmax(pred, dim=1), dim=1))

criterion = LabelSmoothingLoss(smoothing=0.1)

# Mixup function
def mixup_data(x, y, alpha=1.0):
    lam = np.random.beta(alpha, alpha) if alpha > 0 else 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

# Optimizer and scheduler
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)




In [6]:
# Training loop
for epoch in range(30):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Apply Mixup augmentation
        inputs, targets_a, targets_b, lam = mixup_data(images, labels)
        
        # Zero gradients, forward, backward, and optimize
        optimizer.zero_grad()
        outputs, aux_outputs = model(inputs)
        
        # Compute mixup loss with auxiliary classifier
        main_loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)
        aux_loss = mixup_criterion(criterion, aux_outputs, targets_a, targets_b, lam)
        loss = main_loss + 0.4 * aux_loss  # Combine main and auxiliary losses
        loss.backward()
        optimizer.step()
        
        # Track metrics
        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        correct += (lam * predicted.eq(targets_a.data).cpu().sum().float() +
                    (1 - lam) * predicted.eq(targets_b.data).cpu().sum().float())
        total += labels.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = 100. * correct / total
    print(f'Epoch [{epoch+1}/30], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')

    scheduler.step()

   

Epoch [1/30], Loss: 1.7194, Accuracy: 51.84%
Epoch [2/30], Loss: 1.5830, Accuracy: 59.54%
Epoch [3/30], Loss: 1.5561, Accuracy: 60.80%
Epoch [4/30], Loss: 1.4899, Accuracy: 64.52%
Epoch [5/30], Loss: 1.5000, Accuracy: 63.94%
Epoch [6/30], Loss: 1.4799, Accuracy: 64.50%
Epoch [7/30], Loss: 1.4526, Accuracy: 66.00%
Epoch [8/30], Loss: 1.3818, Accuracy: 69.48%
Epoch [9/30], Loss: 1.3418, Accuracy: 70.97%
Epoch [10/30], Loss: 1.3327, Accuracy: 71.33%
Epoch [11/30], Loss: 1.3275, Accuracy: 71.70%
Epoch [12/30], Loss: 1.3147, Accuracy: 72.21%
Epoch [13/30], Loss: 1.2828, Accuracy: 74.16%
Epoch [14/30], Loss: 1.2999, Accuracy: 72.93%
Epoch [15/30], Loss: 1.2847, Accuracy: 73.72%
Epoch [16/30], Loss: 1.2670, Accuracy: 74.57%
Epoch [17/30], Loss: 1.2696, Accuracy: 74.64%
Epoch [18/30], Loss: 1.2758, Accuracy: 74.08%
Epoch [19/30], Loss: 1.2505, Accuracy: 75.30%
Epoch [20/30], Loss: 1.2622, Accuracy: 74.74%
Epoch [21/30], Loss: 1.2559, Accuracy: 74.98%
Epoch [22/30], Loss: 1.2328, Accuracy: 76.0

In [7]:
# Validation
model.eval()
val_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        
        # Only retrieve main output for validation without auxiliary
        outputs = model(images) if not model.training else model(images)[0]  # Main output only
        
        val_loss += criterion(outputs, labels).item() * images.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

val_loss /= len(val_loader.dataset)
val_acc = 100. * correct / total
print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%')


Validation Loss: 0.6777, Validation Accuracy: 88.98%
